## 패키지 설치

In [ ]:
!pip install bs4==0.0.2 requests==2.32.3 langchain-google-genai==1.0.7 streamlit==1.36.0 pytube youtube-transcript-api==0.6.2 lxml

## 유튜브 영상 요약 에이전트

In [ ]:
%%writefile app.py

import re
from urllib.parse import parse_qs, urlparse

import requests
import streamlit as st
from bs4 import BeautifulSoup
from langchain_google_genai import ChatGoogleGenerativeAI
from pytube import YouTube
from youtube_transcript_api import YouTubeTranscriptApi

with st.sidebar:
    st.header("설정")
    api_key = st.text_input("Google API Key를 입력하세요", type="password")


def clean_title(title: str):
    title = re.sub(r"[^\w\s]", "", title)
    title = title.replace(" ", "_")
    return title


def get_youtube_title(video_url):
    response = requests.get(video_url)
    if response.status_code != 200:
        return "Failed to retrieve the page"

    soup = BeautifulSoup(response.text, "html.parser")
    title = soup.find("meta", property="og:title")

    return title["content"] if title else "Title not found"


def get_youtube_info(url: str):
    yt = YouTube(url)
    try:
        title = get_youtube_title(url)
        title = title.replace("/", "")
        thumbnail_url = yt.thumbnail_url
        return title, thumbnail_url
    except Exception as e:
        st.error(f"오류 발생: {str(e)}")
        return None, None


def get_youtube_video_id(url: str):
    if url.startswith("https://youtu.be/"):
        video_id = url.split("/")[-1]
    elif url.startswith("https://www.youtube.com/watch?"):
        parsed_url = urlparse(url)
        query_params = parse_qs(parsed_url.query)
        video_id = query_params.get("v", [None])[0]
    return video_id


st.title("YouTube 영상 요약 서비스")

# 입력 필드
url = st.text_input("YouTube URL을 입력하세요")
watch_script = st.checkbox("원문 스크립트 포함")

# 모델 선택
model_options = ["gemini-1.5-pro-002", "gemini-1.5-flash-002", "gemini-1.5-flash-8B", "gemini-1.5-pro", "gemini-1.5-flash"]
selected_model = st.selectbox("모델을 선택하세요", model_options)

if st.button("요약하기") and url:
    with st.spinner("영상을 분석 중입니다..."):
        # 모델 설정
        model = selected_model
        llm = ChatGoogleGenerativeAI(model=model, google_api_key=api_key)

        # 영상 정보 추출
        title, thumbnail_url = get_youtube_info(url)

        if title and thumbnail_url:
            st.image(thumbnail_url, caption=title)

            # 자막 추출
            video_id = get_youtube_video_id(url)
            try:
                srt = YouTubeTranscriptApi.get_transcript(video_id, languages=["ko"])
                content = [i for i in srt if i["text"] != "[음악]"]

                # 프롬프트 생성
                base_script = """### 명령어
내가 만든 스타일대로 markdown 양식으로 변환해줘.

### 요구사항
- timestamp_summary는 적절하게 타임스탬프를 큰 내용 단위로 나눠줘.
- timestamp_summary는 정적인 분위기가 아닌, 얘기를 하듯이 작성해줘.
- timestamp_summary는 반드시 한글로 작성해줘.
- tags는 이 영상을 분류하기 위한 목적으로 만들어줘,
- tgas는 최대 5개까지만 만들어줘,
- 타임라인은 5분 미만의 영상의 경우에는 최소 3개의 타임라인이 나와야 하고 10분 미만의 영상의 경우에는 최소 6개의 타임라인이 나와야 합니다. 30분 이상의 영상의 경우에는 최소 10분당 2개 이상의 타임라인이 만들어져야 합니다.
- title 들을 적을 땐, 가능하면 title의 앞 부분에 해당 요약 내용에 어울리는 이모티콘을 하나 배정해줘.
- 타임라인에 타임스탬프는 해당 유튜브 영상의 위치로 클릭 해서 이동할 수 있게 t 파라미터를 추가해서 해당 타임 스탬프의 초 단위 값을 넣어줘.

### 양식
---
tags:
- {tag_name_1}
- {tag_name_2}
- ...
url: {youtube_url}
---
# 핵심주제
<details open>
<summary>{timeline_summary_title_1}</summary>
<ul>
<li>...</li>
...
</ul>
</details>

# 타임라인
[[{hh:mm:ss}]]({youtube_url}&t={second}) {timestamp_summary_title_1}
- {timestamp_summary_1_content}
- ...
{more}
...""".replace(
                    "{youtube_url}", url
                )

                if watch_script:
                    base_script = base_script.replace(
                        "{more}",
                        """<details open>
<summary>원문 스크립트 보기</summary>
{original_script}
</details>""",
                    )
                else:
                    base_script = base_script.replace("{more}\n", "")

                messages = [
                    ("system", base_script),
                    ("human", f"### 입력 데이터\n{content}\n### 출력\n"),
                ]

                result = llm.invoke(messages)
                if result.content:
                    st.markdown(result.content)

                    # 결과 파일 저장
                    if st.button("결과 저장하기"):
                        with open(f"{title}.md", mode="w", encoding="utf-8") as f:
                            f.write(result.content)
                        st.success("파일이 저장되었습니다!")
                else:
                    st.error("요약 생성에 실패했습니다.")

            except Exception as e:
                st.error(f"자막 추출 중 오류가 발생했습니다: {str(e)}")


# 코랩 ip 확인

In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

# 포트 터널링 node 패키지 설치

In [ ]:
!npm install localtunnel

# 웹 프로그램 실행

In [ ]:
!streamlit run app.py &>/content/logs.txt &

# 포트 개방

In [ ]:
!npx localtunnel --port 8501